# Planet to planet low-thrust

In this tutorial we show the use of the {class}`pykep.trajopt.direct_pl2pl` to find a low-thrust (zero-order hold continous thrust) trajectory connecting two moving planets. 

The decision vector for this class, compatible with pygmo {cite:p}`pagmo` UDPs (User Defined Problems), is:

$$
\mathbf x = [t_0, m_f, V_{sx}^\infty, V^\infty_{sy}, V^\infty_{sz}, V^\infty_{fx}, V^\infty_{fy}, V^\infty_{fz}, u_{x0}, u_{y0}, u_{z0}, u_{x1}, u_{y1}, u_{z1}, ..., T_{tof}]
$$

containing the starting epoch $t_0$ as a MJD2000, the final mass $m_f$ as well as the starting and final $V^{\infty}$, throttles and the time-of-flight $T_{tof}$.

:::{note}
This notebook makes use of the commercial solver SNOPT 7 and to run needs a valid `snopt_7_c` library installed in the system. In case SNOPT7 is not available, you can still run the notebook using, for example `uda = pg.algorithm.nlopt("slsqp")` with minor modifications.

Basic imports:

In [1]:
import pykep as pk
import numpy as np
import time
import pygmo as pg
import pygmo_plugins_nonfree as ppnf
import time

from matplotlib import pyplot as plt

We start defining the problem data. For the purpose of this simple notebook we choose a simple Earth to Mars transfer.

In [2]:
# Problem data

####
## Testcase Earth-Mars with
mu = pk.MU_SUN
max_thrust = 0.3
isp = 3000
veff = isp * pk.G0
tof = 550.0

posvel0 = [
    [-125036811000.422, -83670919168.87277, 2610252.8064399767],
    [16081.829029183446, -24868.923007449284, 0.7758272135425942]
]
posvelf = [
    [-169327023332.1986, -161931354587.78766, 763967345.9733696],
    [17656.297796509956, -15438.116653052988, -756.9165272457421]
]

# Define initial and target
p1 = pk.planet(pk.udpla.keplerian(when=pk.epoch(0), posvel = posvel0, mu_central_body=mu))
p2 = pk.planet(pk.udpla.keplerian(when=pk.epoch(tof), posvel = posvelf, mu_central_body=mu))

# Initial state
ms = 1500.0

# Number of segments
nseg = 4


In [3]:
# # Normalise
# L = pk.AU
# TIME = np.sqrt(L**3 / mu)
# VEL = L / TIME
# ACC = VEL / TIME
# MASS = ms

# print(f'Normalise L {L:.4f} T {TIME:.4f} V {VEL:.4f} M {MASS:.4f}')

# # Problem data
# mu = mu / (L**3 / TIME**2)
# max_thrust = max_thrust / (MASS * L / TIME**2)
# veff = veff / VEL

# # Initial state
# ms = ms / MASS
# posvel0[0] = (np.array(posvel0[0]) / L).tolist()
# posvel0[1] = (np.array(posvel0[1]) / VEL).tolist()

# # Final state
# posvelf[0] = (np.array(posvelf[0]) / L).tolist()
# posvelf[1] = (np.array(posvelf[1]) / VEL).tolist()

# # tof
# tof = tof / TIME

We here instantiate two different versions of the same UDP (User Defined Problem), with analytical gradients and without. 

For the purpose of this simple notebook we choose a relatively simple Earth to Mars transfer with an initial $V_{\infty}$ of 3 km/s.

In [4]:
udp_g = pk.trajopt.direct_pl2pl(
        pls=p1,
        plf=p2,
        ms=ms,
        mu=mu,
        max_thrust=max_thrust,
        veff=veff,
        t0_bounds=[0.0, 0.0],
        tof_bounds=[tof,tof],
        mf_bounds=[ms*0.5, ms],
        vinfs=0.,
        vinff=0.,
        nseg=nseg,
        cut=0.6,
        mass_scaling=ms,
        r_scaling=pk.AU,
        v_scaling=pk.EARTH_VELOCITY,
        with_gradient=False,
        high_fidelity=False
)

## Analytical performances of the analytical gradient

And we take a quick look at the performances of the analytical gradient with respect to the numerically computed one.

In [5]:
# We need to generste a random chromosomes compatible with the UDP where to test the gradient.
prob_g = pg.problem(udp_g)
pop_g = pg.population(prob_g, 1)

In [6]:
def compute_numerical_gradient(sf_leg, sf_leg_type = 'lf'):
    import numpy as np
    import pykep as pk
    import pygmo as pg

    state_length = np.array(sf_leg.rvs).flatten().size + 1
    throttle_length = np.array(sf_leg.throttles).size
    chromosome = np.zeros((state_length * 2 + throttle_length + 1))
    chromosome[0:state_length] = np.append(np.array(sf_leg.rvs).flatten(), sf_leg.ms)
    chromosome[state_length:state_length+throttle_length] = np.array(sf_leg.throttles)
    chromosome[state_length+throttle_length:state_length*2+throttle_length] = np.append(np.array(sf_leg.rvf).flatten(), sf_leg.mf)
    chromosome[-1] = sf_leg.tof

    def set_and_compute_constraints(chromosome, sf_leg_type = 'lf'):

        if sf_leg_type == 'hf' or sf_leg_type == 'high-fidelity':
            sf_leg_constraint = pk.leg.sims_flanagan_hf()
        else:
            sf_leg_constraint = pk.leg.sims_flanagan()
        sf_leg_constraint.cut = 0.5
        sf_leg_constraint.max_thrust = 1
        sf_leg_constraint.mu = 0.012
        sf_leg_constraint.veff = 1
        sf_leg_constraint.rvs = [chromosome[0:3],chromosome[3:6]]
        sf_leg_constraint.ms = chromosome[6]
        sf_leg_constraint.throttles = chromosome[state_length:state_length+throttle_length]
        sf_leg_constraint.rvf = [chromosome[state_length+throttle_length:state_length+throttle_length+3],chromosome[state_length+throttle_length+3:state_length+throttle_length+6]]
        sf_leg_constraint.mf = chromosome[2*state_length+throttle_length-1]
        sf_leg_constraint.tof = chromosome[2*state_length+throttle_length]
        eq_con = sf_leg_constraint.compute_mismatch_constraints()
        ineq_con = sf_leg_constraint.compute_throttle_constraints()
        return np.concatenate((eq_con, ineq_con))

    return pg.estimate_gradient_h(callable = lambda x : set_and_compute_constraints(x, sf_leg_type), x=chromosome)

In [7]:
def test_mc_grad_hf():
    import numpy as np

    sf_leg = pk.leg.sims_flanagan_hf()
    sf_leg.cut = 0.5
    sf_leg.throttles = np.array([0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24,
    0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34])
    sf_leg.rvs = np.array([[1, 0.1, -0.1], [0.2, 1.0, -0.2]])
    sf_leg.ms = 1
    sf_leg.rvf = np.array([[1.2, -0.1, 0.1], [-0.2, 1.023, -0.44]])
    sf_leg.mf = 13 / 15
    sf_leg.max_thrust = 1
    sf_leg.mu = 0.012
    sf_leg.veff = 1
    sf_leg.tof = 1
    state_length = np.array(sf_leg.rvs).flatten().size + 1
    throttle_length = np.array(sf_leg.throttles).size

    num_grad = compute_numerical_gradient(sf_leg, sf_leg_type = 'hf')
    num_grad = num_grad.reshape((17, 45), order='C')
    grad_rvm, grad_rvm_bck, grad_final = sf_leg.compute_mc_grad()
    a_tc_grad = sf_leg.compute_tc_grad()
    a_grad = np.zeros((state_length+throttle_length // 3, 2 * state_length + throttle_length + 1))
    a_grad[0:state_length, 0:state_length] = grad_rvm
    a_grad[0:state_length, state_length:state_length + throttle_length] = grad_final[:,0:throttle_length] 
    a_grad[0:state_length, state_length+throttle_length:state_length*2+throttle_length] = grad_rvm_bck
    a_grad[0:state_length, state_length*2+throttle_length] = grad_final[:, throttle_length:throttle_length + 1].reshape(7,)
    a_grad[state_length:, state_length:state_length+throttle_length] = a_tc_grad
    return np.allclose(num_grad, a_grad, atol=1e-8)

In [8]:
test_mc_grad_hf()

True

In [9]:
# Check gradientss
# Analytical
an_grad = udp_g.gradient(pop_g.champion_x)
# Numerical
num_grad = pg.estimate_gradient_h(udp_g.fitness, pop_g.champion_x, dx=1e-12)

# Suppose sparsity gives the positions (flat indices in num_grad)
sparsity = udp_g.gradient_sparsity()  # e.g. [0, 3, 10, ...]

# Make an empty dense gradient
dim = int(9+3*nseg)
dense_ana_grad = np.zeros_like(num_grad).reshape(int(len(num_grad)/dim),dim)
num_grad = num_grad.reshape(int(len(num_grad)/dim),dim)

# Fill in analytical entries at the right positions
for jj in range(len(an_grad)):
    dense_ana_grad[sparsity[jj][0], sparsity[jj][1]] = an_grad[jj]
    diff_tmp = abs(dense_ana_grad[sparsity[jj][0], sparsity[jj][1]] - num_grad[sparsity[jj][0], sparsity[jj][1]])
    if diff_tmp > 1e-3:
        print('sparsity[jj]',sparsity[jj] , 'analytical', dense_ana_grad[sparsity[jj][0], sparsity[jj][1]],'numerical', num_grad[sparsity[jj][0], sparsity[jj][1]],'diff',diff_tmp)
# dense_ana_grad = dense_ana_grad.reshape(-1)
# num_grad = num_grad.reshape(-1)

# Now you can compare
diff_bool = np.allclose(num_grad, dense_ana_grad, atol=1e-3, rtol=1e-3)
diff = num_grad - dense_ana_grad
# print('Analytical', dense_ana_grad)
# print('Numerical', num_grad)
print('')
print('diff_bool', diff_bool)
print('len J', 10 + nseg)
print('len x', 9 + 3*nseg)
print('shape(diff)', np.shape(diff))
print("‖diff‖:", np.linalg.norm(diff))

sparsity[jj] [1, 16] analytical 0.000440602010096 numerical -0.00186887542479 diff 0.00230947743488
sparsity[jj] [1, 18] analytical 0.137563737148 numerical 0.139540231222 diff 0.00197649407345
sparsity[jj] [2, 2] analytical 0.000251486378784 numerical -0.00125455201783 diff 0.00150603839661
sparsity[jj] [2, 3] analytical -0.000497611631463 numerical 0.000651330841113 diff 0.00114894247258
sparsity[jj] [2, 9] analytical -0.0687659348703 numerical -0.0698737364265 diff 0.00110780155624
sparsity[jj] [4, 2] analytical -0.000188589419457 numerical 0.00104430353254 diff 0.001232892952
sparsity[jj] [4, 3] analytical 0.000483380342892 numerical -0.000586567831344 diff 0.00106994817424
sparsity[jj] [5, 2] analytical 0.000200072087443 numerical -0.00113057711341 diff 0.00133064920085
sparsity[jj] [5, 16] analytical 0.00374621588428 numerical 0.00260532336445 diff 0.00114089251983

diff_bool False
len J 14
len x 21
shape(diff) (14, 21)
‖diff‖: 0.00555503790336


In [10]:
xxx

NameError: name 'xxx' is not defined

First the analytical gradient:

In [ ]:
%%timeit
udp_g.gradient(pop_g.champion_x)

Then a simple numerical gradient based on finite differences:

In [ ]:
%%timeit
pg.estimate_gradient(udp_g.fitness, pop_g.champion_x)

Then a higher order numerical gradient:

In [ ]:
%%timeit
pg.estimate_gradient_h(udp_g.fitness, pop_g.champion_x)

The analytical gradient its exact and faster, seems like a no brainer to use it. 

In reality, the effects on the optimization technique used are not straightforward, making the option to use numerical gradients still interesting in some, albeit rare, cases.

## Solving the low-thrust transfer

We define (again) the optimization problem, and set a tolerance for *pagmo* to be able to judge the relative value of two individuals. 

:::{note}
This tolerance has a different role from the numerical tolerance set in the particular algorithm chosen to solve the problem and is only used by the *pagmo* machinery to decide outside the optimizer whether the new proposed indivdual is better than what was the previous *champion*.

In [ ]:
prob_g = pg.problem(udp_g)
prob_g.c_tol = 1e-6

... and we define an optimization algorithm.

In [ ]:
snopt72 = "/Users/dario.izzo/opt/libsnopt7_c.dylib"
uda = ppnf.snopt7(library=snopt72, minor_version=2, screen_output=False)
uda.set_integer_option("Major iterations limit", 2000)
uda.set_integer_option("Iterations limit", 20000)
uda.set_numeric_option("Major optimality tolerance", 1e-3)
uda.set_numeric_option("Major feasibility tolerance", 1e-11)

algo = pg.algorithm(uda)

In [ ]:
# uda = pg.nlopt("slsqp")
# algo = pg.algorithm(uda)

In [ ]:
# ip = pg.ipopt()
# ip.set_numeric_option("tol", 1E-9) # Change the relative convergence tolerance
# ip.set_integer_option("max_iter", 500) # Change the maximum iterations
# ip.set_integer_option("print_level", 0) # Makes Ipopt unverbose
# ip.set_string_option("nlp_scaling_method", "none") # Removes any scaling made in auto mode
# ip.set_string_option("mu_strategy", "adaptive") # Alternative is to tune the initial mu value
# algo = pg.algorithm(ip)

We solve the problem from random initial guess ten times and only save the result if a feasible solution is found (as defined by the criterias above)

In [ ]:
masses = []
xs = []
for i in range(10):
    pop_g = pg.population(prob_g, 1)
    pop_g = algo.evolve(pop_g)
    if(prob_g.feasibility_f(pop_g.champion_f)):
        print(".", end="")
        masses.append(pop_g.champion_x[1])
        xs.append(pop_g.champion_x)
        break
    else:
        print("x", end ="")
print("\nBest mass is: ", np.max(masses))
print("Worst mass is: ", np.min(masses))
best_idx = np.argmax(masses)

And we plot the trajectory found:

In [ ]:
udp_g.pretty(xs[best_idx])

In [ ]:
ax = udp_g.plot(xs[best_idx], show_gridpoints=True)
ax.view_init(90, 0)